<a href="https://colab.research.google.com/github/tainatrajano/UCDPA_TainaAraujoTrajano/blob/main/rainfall_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Libraries & Functions

In [ ]:
from pandas.core.groupby.base import dataclasses
###Remove what will not be used

import pandas as pd
import numpy as np
import re
import requests

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_curve, classification_report, confusion_matrix, mean_squared_error
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge

#calculates the func of each row and returns on a list #####REVISE
def func_by_array(df, list_cols, func):
  list_rows = []
  i = 0
  for i in range(0,len(df.index)):
    x = func(df[list_cols].iloc[i].dropna())
    list_rows.append(x)

  return list_rows

#calculates ECDF for a one-dimensional array###REVISE
def ecdf(data):
    n = len(data)
    x = np.sort(data)
    y = np.arange(1, n + 1) / n

    return x, y


2. Importing Data

In [5]:
#importing csv file downloaded from https://www.met.ie/climate/available-data/historical-data website on 30/09/2023
dRainfall_df = pd.read_csv("/content/drive/MyDrive/database/dublin_rainfall.csv")
kRainfall_df = pd.read_csv("/content/drive/MyDrive/database/kerry_rainfall.csv")

#importing data from https://data.gov.ie/dataset/monthly-weather-valentia-observatory to compare data from both sources from 2020-2022 for county Kerry, only mean temperature and rainfall
kMonthlyWeather_df = pd.DataFrame()
try:
  get_API = requests.get('https://prodapi.metweb.ie/monthly-data/Valentia%20Observatory')
  kMonthlyWeather_df = pd.DataFrame(get_API.json())

except:
  print("API failed to retrieve data.")


ParserError: ignored

3. Analysing data

In [ ]:
#using method .join() to left join second DataFrame into the first using columns 'cca3', 'Country Code' as index: df
if not dfCountryRegion.empty:
  df = wdiDf.join(dfCountryRegion.set_index('cca3'), on='Country Code')

  #removimg the " [YRYYYY]" part of the columns' names using Regex .sub() method, a for loop used to run through columns' names (iterator) and result append to a list
  new_headers = []
  for s in df.columns:
    new_headers.append(re.sub(r"\s.YR\d{4}.","",s))

  df.columns = new_headers
  years_list = new_headers[4:len(df.columns)-1]
  countries_list = df['Country Name'].unique()
  regions_list = df['region'].unique()
  series_list = df['Series Name'].unique()
  print(df.info())

else:
    print("DataFrame empty.")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555 entries, 0 to 554
Data columns (total 66 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Country Name   555 non-null    object 
 1   Country Code   555 non-null    object 
 2   Series Name    555 non-null    object 
 3   Series Code    555 non-null    object 
 4   1961 [YR1961]  91 non-null     float64
 5   1962 [YR1962]  95 non-null     float64
 6   1963 [YR1963]  95 non-null     float64
 7   1964 [YR1964]  95 non-null     float64
 8   1965 [YR1965]  95 non-null     float64
 9   1966 [YR1966]  99 non-null     float64
 10  1967 [YR1967]  102 non-null    float64
 11  1968 [YR1968]  103 non-null    float64
 12  1969 [YR1969]  105 non-null    float64
 13  1970 [YR1970]  102 non-null    float64
 14  1971 [YR1971]  108 non-null    float64
 15  1972 [YR1972]  108 non-null    float64
 16  1973 [YR1973]  108 non-null    float64
 17  1974 [YR1974]  108 non-null    float64
 18  1975 [YR19

In [ ]:
#considering the data missing is MAR type, KKNImputer will be used to treat the missing data considering values of the region for that indicator ####REVIEW
  #compute the np.mean of each row prior the KKNImputer
mean_before = func_by_row(df, years_list, np.mean)
  # define imputer
imputer = KNNImputer(n_neighbors=6, weights='uniform', metric='nan_euclidean')
imputed_df = pd.DataFrame([], columns=df.columns)
i = 0
df_nomissingvalues = df
for series in series_list:
  for region in regions_list:
    filtered_df = df[(df['Series Name'] == series) & (df['region'] == region)]
    numerical_columns = filtered_df[years_list]
    # Impute missing values using KNNImputer
    imputed_data = imputer.fit_transform(numerical_columns)

    # Create a new DataFrame with imputed values
    imputed_df[years_list] = pd.DataFrame(imputed_data,years_list)

  #compute the np.mean of each row after the KKNImputer
mean_after = func_by_row(imputed_df, years_list, np.mean)

print("mean before: {}".format(mean_before))
print("mean after: {}".format(mean_after))

ValueError: ignored

In [ ]:
#Visualizations ####REViEW
_ = sns.lineplot(mean_before, x = years_list, y = [-50,50], colour="red")
_ = sns.lineplot(mean_after, x = years_list, y = [-50,50], colour="blue")
_ = plt.xlabel("No of Row")
plt.show()

4. Machine Learning

In [ ]:
#using LinearRegression to observed if Okun's Law can be supported by this DataFrame, the higher the GDP the lower the unemployment rate, ####REVIEW
  #also to observe if the government effectiviness is negative related to the unemployment rate
reg = LinearRegression()
for row in df.rows:
  row = row[4:]
  reg.fit(row,[-1,1])
  predictions = reg.predict(row,[-1,1])

  # Data Splitting
X = df[['feature1', 'feature2']]  # Features
y = df['target_column']  # Target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # Linear Regression Model
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

  # Model Evaluation
mse = mean_squared_error(y_test, y_pred)


5. Visualise

In [ ]:
sns.scatterplot(x=years_list, y='feature2', data=df) ####REVIEW
plt.title('World Bank Indicators')
plt.xlabel('Years')
plt.show()